In [2]:
import pandas as pd

In [17]:
# Load dataframes
current = pd.read_csv('data/Current_Sheet.csv')
emissions_limited = pd.read_csv('data/Emissions_Limited_2040.csv')
business_as_usual = pd.read_csv('data/Business_as_Usual_2080.csv')
bau_plus_1degree = pd.read_csv('data/BAU_plus_1degree_2080.csv')

In [32]:
def compare_taxon_lists(df1, df2, Taxon="Taxon"):
    """
    Compare the taxon lists of two datasets.

    Parameters:
    df1 (pd.DataFrame): First dataframe to compare.
    df2 (pd.DataFrame): Second dataframe to compare.
    
    Returns:
    dict: A dictionary with keys 'unique_to_df1', 'unique_to_df2', and 'common' 
          containing taxon lists unique to each dataframe and common to both.
    """
    #remove duplicates
    taxon_df1 = set(df1[Taxon].dropna().unique())
    taxon_df2 = set(df2[Taxon].dropna().unique())

    #compare dataframes
    unique_to_df1 = list(taxon_df1 - taxon_df2)
    unique_to_df2 = list(taxon_df2 - taxon_df1)
    common = list(taxon_df1 & taxon_df2)
    
    return {
        'unique_to_df1': unique_to_df1,
        'unique_to_df2': unique_to_df2,
        'common': common
    }


In [40]:
result = compare_taxon_lists(current, business_as_usual)
print(f"""Test results:
Unique to df1 ({len(result['unique_to_df1'])}) 
Unique to df2 ({len(result['unique_to_df2'])})
Common ({len(result['common'])})
""")

Test results:
Unique to df1 (0) 
Unique to df2 (0)
Common (3829)



In [50]:
#result['common']

['Romneya coulteri',
 'Gladiolus saundersii',
 'Iris innominata',
 'Rhododendron qiaojiaense',
 'Saxifraga hostii',
 'Magnolia nitida',
 'Acer campbellii',
 'Colutea istria',
 'Uvularia grandiflora',
 'Ilex collina',
 'Ugni molinae',
 'Cyclamen hederifolium',
 'Jasione laevis',
 'Rosa serafinii',
 'Galanthus reginae-olgae',
 'Salix lapponum',
 'Lilium columbianum',
 'Dictamnus albus',
 'Dasiphora fruticosa',
 'Ozothamnus leptophyllus',
 'Alchemilla lithophila',
 'Mirabilis multiflora',
 'Crambe kotschyana',
 'Fessia greilhuberi',
 'Haplocarpha scaposa',
 'Rhododendron selense',
 'Fraxinus latifolia',
 'Yucca baccata',
 'Magnolia stellata',
 'Eucalyptus perriniana',
 'Actinidia rufa',
 'Akebia quinata',
 'Sedum obtusatum',
 'Celtis laevigata',
 'Phlomis anisodonta',
 'Pinus brutia',
 'Ludwigia alternifolia',
 'Phlox douglasii',
 'Primula polyneura',
 'Litsea pungens',
 'Vitis rotundifolia',
 'Koeleria vallesiana',
 'Juniperus deppeana',
 'Kageneckia oblonga',
 'Glycyrrhiza glabra',
 'Ae